In [13]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

In [14]:
mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)

True

In [15]:
symbol = "USDCAD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime(2024, 10, 8)))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()


def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Function to add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Indicators
# Calculate indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

df = add_rolling_features(df, window=5)
df = add_lag_features(df, lags=[1, 2, 3, 4, 5])

df = df.dropna().reset_index(drop=True)

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

""" csv_file_path = 'ICMT5EURUSD2010_3112_D1.csv'  # Specify your desired path
df.to_csv(csv_file_path, index=False)
df """

" csv_file_path = 'ICMT5EURUSD2010_3112_D1.csv'  # Specify your desired path\ndf.to_csv(csv_file_path, index=False)\ndf "

In [ ]:
""" 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Visual Inspection: Plot the time series data
plt.figure(figsize=(14, 7))
plt.plot(df['time'], df['close'], label='Close Price')
plt.title('GBPUSD Close Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

# Summary Statistics
print(df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']].describe())

# Outlier Detection using IQR
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))
    return outliers

outliers = df[['open', 'high', 'low', 'close']].apply(detect_outliers_iqr)
print(outliers.sum())  # Count of outliers in each column

# Plotting boxplots to visualize outliers
plt.figure(figsize=(14, 7))
sns.boxplot(data=df[['open', 'high', 'low', 'close']])
plt.title('Boxplot of OHLC Prices')
plt.show()
 """

In [16]:
StopLoss = 2
TakeProfit = 3
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)
#print('BreatEvenRatio:', BreakEvenRatio)

Mean Candle: 0.00637524509803922


In [17]:
# Calculate total number of 1s in b_flag and s_flag columns
total_b_flags = df['b_flag'].sum()
total_s_flags = df['s_flag'].sum()

# Total number of rows in the DataFrame
total_rows = len(df)

# Calculate counts in segments of complete 100% data
count_100_b_flags = total_b_flags
count_100_s_flags = total_s_flags

# Calculate counts in intervals of 10%
interval_counts = []
for i in range(0, 101, 10):
    start_idx = int(i / 100 * total_rows)
    end_idx = int((i + 10) / 100 * total_rows)
    
    interval_b_flags = df['b_flag'].iloc[start_idx:end_idx].sum()
    interval_s_flags = df['s_flag'].iloc[start_idx:end_idx].sum()
    
    interval_counts.append((f'{i}% - {i+10}%', interval_b_flags, interval_s_flags))

# Print results
print("Total number of 1s:")
print(f"b_flag: {total_b_flags}")
print(f"s_flag: {total_s_flags}")

print("\nCounts in segments of 100% data:")
print(f"b_flag: {count_100_b_flags}")
print(f"s_flag: {count_100_s_flags}")

print("\nCounts in intervals of 10%:")
for interval, count_b, count_s in interval_counts:
    print(f"{interval}: b_flag={count_b}, s_flag={count_s}")

Total number of 1s:
b_flag: 81
s_flag: 24

Counts in segments of 100% data:
b_flag: 81
s_flag: 24

Counts in intervals of 10%:
0% - 10%: b_flag=0, s_flag=20
10% - 20%: b_flag=15, s_flag=2
20% - 30%: b_flag=19, s_flag=0
30% - 40%: b_flag=14, s_flag=0
40% - 50%: b_flag=21, s_flag=0
50% - 60%: b_flag=11, s_flag=2
60% - 70%: b_flag=1, s_flag=0
70% - 80%: b_flag=0, s_flag=0
80% - 90%: b_flag=0, s_flag=0
90% - 100%: b_flag=0, s_flag=0
100% - 110%: b_flag=0, s_flag=0


In [18]:
# Feature extraction
df.drop(columns=['s_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['b_flag'])
X_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]

correlation_matrix = X_df.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_correlation_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]
X_df = X_df.drop(columns=high_correlation_features)

# Get the list of selected feature names
selected_feature_names_X = list(X_df.columns)

# Combine lists if you need a single list for all selected features

print(selected_feature_names_X )

Feature Extraction: 100%|██████████| 40/40 [00:04<00:00,  8.15it/s]


" \n# Ensure b_flag is at the end after feature selection\nX_df = select_features(X, X['b_flag'])\nX_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]\n\ncorrelation_matrix = X_df.corr().abs()\nupper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))\nhigh_correlation_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]\nX_df = X_df.drop(columns=high_correlation_features)\n\n# Get the list of selected feature names\nselected_feature_names_X = list(X_df.columns)\n\n# Combine lists if you need a single list for all selected features\n\nprint(selected_feature_names_X ) "

In [ ]:
""" from scipy import stats

# Removing outliers using Z-score
z_scores = stats.zscore(X_df.select_dtypes(include=[float, int]))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
X_df_clean = X_df[filtered_entries]

print("Data shape after removing outliers:", X_df_clean.shape)

from sklearn.preprocessing import StandardScaler

# Standardizing the features
scaler = StandardScaler()
X_df_scaled = X_df_clean.copy()
X_df_scaled.iloc[:, :-1] = scaler.fit_transform(X_df_scaled.iloc[:, :-1])

print("Data after scaling:")
print(X_df_scaled.head())

# Visualizing the data
plt.figure(figsize=(14, 7))
for feature in X_df_scaled.columns[:-1]:  # Exclude the 'b_flag'
    plt.plot(X_df_scaled.index, X_df_scaled[feature], label=feature)
plt.title('Selected Features Over Time')
plt.xlabel('Time')
plt.ylabel('Feature Values')
plt.legend()
plt.show()

# Analyzing the distribution of the selected features
X_df_scaled.hist(bins=50, figsize=(20, 15))
plt.suptitle('Histogram of Selected Features')
plt.show()
import seaborn as sns

# Plotting the distribution of each feature
for feature in X_df_scaled.columns[:-1]:  # Exclude the 'b_flag'
    plt.figure(figsize=(14, 7))
    sns.distplot(X_df_scaled[feature], bins=50)
    plt.title(f'Distribution of {feature}')
    plt.show()

# Checking for anomalies using boxplots again
plt.figure(figsize=(14, 7))
sns.boxplot(data=X_df_scaled.iloc[:, :-1])  # Exclude the 'b_flag'
plt.title('Boxplot of Selected Features After Scaling')
plt.show()
# Example: Removing values beyond a certain z-score threshold
threshold = 3
z_scores = np.abs(stats.zscore(X_df_scaled.select_dtypes(include=[float, int])))
X_df_cleaned = X_df_scaled[(z_scores < threshold).all(axis=1)]

print("Data shape after removing noise:", X_df_cleaned.shape)
# Summary Statistics
print(X_df_cleaned.describe())

# Correlation Analysis
plt.figure(figsize=(14, 7))
correlation_matrix = X_df_cleaned.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Cleaned and Scaled Features')
plt.show()

# Feature Importance (if applicable)
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(X_df_cleaned.drop('b_flag', axis=1), X_df_cleaned['b_flag'])

# Feature importance
# importances = model.feature_importances_
# feature_names = X_df_cleaned.columns[:-1]  # Exclude the 'b_flag'
# indices = np.argsort(importances)[::-1]

# Plotting the feature importance
# plt.figure(figsize=(14, 7))
# plt.title("Feature Importance")
# plt.bar(range(X_df_cleaned.shape[1] - 1), importances[indices], align="center")
# plt.xticks(range(X_df_cleaned.shape[1] - 1), feature_names[indices], rotation=90)
# plt.xlim([-1, X_df_cleaned.shape[1] - 1])
# plt.show()
 """

In [ ]:
""" 
# Rechecking for any remaining noise
plt.figure(figsize=(14, 7))
for feature in X_df_scaled.columns[:-1]:  # Exclude the 'b_flag'
    sns.distplot(X_df_scaled[feature], bins=50, kde=True)
    plt.title(f'Distribution of {feature}')
    plt.show()
# Rechecking summary statistics
print(X_df_clean.describe())

# Rechecking correlation
plt.figure(figsize=(14, 7))
correlation_matrix = X_df_clean.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Cleaned Features')
plt.show()

from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Assuming you have a target variable 'b_flag'
X_features = X_df_clean.drop('b_flag', axis=1)
y_target = X_df_clean['b_flag']

# Train a RandomForestClassifier to get feature importances
model = RandomForestClassifier()
model.fit(X_features, y_target)

# Feature importance
importances = model.feature_importances_
feature_names = X_features.columns
indices = np.argsort(importances)[::-1]

 # Plotting the feature importance
plt.figure(figsize=(14, 7))
plt.title("Feature Importance")
plt.bar(range(X_features.shape[1]), importances[indices], align="center")
plt.xticks(range(X_features.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X_features.shape[1]])
plt.show() 
"""

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from joblib import dump
import os
import pandas as pd
import numpy as np

sum_fp = 0
sum_tp = 0

# Ensure the time series order is preserved in the split
split = int(0.80 * len(X_df))
train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['b_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['b_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('USDCAD_D1_3112_BuyNEW', exist_ok=True)
dump(sc_mt, 'USDCAD_D1_3112_BuyNEW/scaler.joblib')

# Initialise RandomForestClassifier with fixed hyperparameters
rf_classifier_mt = RandomForestClassifier(
    n_estimators=120,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='log2',
    class_weight={0: 10, 1: 5},
    random_state=10

)

# Train Model
rf_classifier_mt.fit(x_train, y_train)

dump(rf_classifier_mt, 'USDCAD_D1_3112_BuyNEW/model.joblib')

# Predict on training data
y_train_pred = rf_classifier_mt.predict(x_train)

print("Confusion Matrix (Training Data):")
cm_train = confusion_matrix(y_train, y_train_pred)
print(cm_train)

train_false_positives = cm_train[0][1]
train_true_positives = cm_train[1][1]

train_precision = precision_score(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, rf_classifier_mt.predict_proba(x_train)[:, 1])

print('Training Data Results:')
print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
print('False Positives:', train_false_positives)
print('True Positives:', train_true_positives)
print('Precision:', train_precision)
print('Accuracy:', train_accuracy)
print('Recall:', train_recall)
print('F1 Score:', train_f1)
print('ROC AUC:', train_roc_auc)
print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

# Predict on testing data
y_test_pred = rf_classifier_mt.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, rf_classifier_mt.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


Confusion Matrix (Training Data):
[[1619    0]
 [ 279  889]]
Training Data Results:
WIN/LOSS-Diff: 60.0 %
False Positives: 0
True Positives: 889
Precision: 1.0
Accuracy: 0.8998923573735199
Recall: 0.7611301369863014
F1 Score: 0.8643655809431211
ROC AUC: 0.9993336830615889
Ratio Total: 100.0
BreakEvenRatio: 0.4
____________________________________________________________________________________________________________________________
Confusion Matrix (Testing Data):
[[356  10]
 [301  30]]
Testing Data Results:
WIN/LOSS-Diff: 35.0 %
False Positives: 10
True Positives: 30
Precision: 0.75
Accuracy: 0.5538020086083214
Recall: 0.09063444108761329
F1 Score: 0.16172506738544473
ROC AUC: 0.6517425255476863
Ratio Total: 75.0
BreakEvenRatio: 0.4
____________________________________________________________________________________________________________________________


In [20]:
import json
from joblib import load

buyscaler = load('USDCAD_D1_3112_BuyNEW/scaler.joblib')
buymodel = load('USDCAD_D1_3112_BuyNEW/model.joblib')
with open('USDCAD_D1_3112_BuyNEW/feature_names.json', 'r') as f:
        feature_names = json.load(f)

buy_features = feature_names[:-1] 

df_new = X[buy_features]

In [21]:
scaled = buyscaler.transform(df_new)
y_pred =buymodel.predict(scaled)

,open,WILLR_42__mean_second_derivative_central,WILLR_15__mean_second_derivative_central,"WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","WILLR_15__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","WILLR_15__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","WILLR_42__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""mean""","WILLR_15__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""",WILLR_15,"WILLR_15__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""mean""",...,"WILLR_42__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","WILLR_42__linear_trend__attr_""stderr""","WILLR_15__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","WILLR_15__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","WILLR_42__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0","WILLR_15__fft_coefficient__attr_""imag""__coeff_3","WILLR_42__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.0","WILLR_15__fft_coefficient__attr_""abs""__coeff_5","WILLR_15__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0",WILLR_15__autocorrelation__lag_5
time,,,,,,,,,,,,,,,,,,,,,
2023-11-24,1.36932,-1.845411,-2.203622,1.975411,7.217288,66.931515,-0.047990,0.298664,-85.276074,0.049459,...,-4.872647,0.948949,336.523046,-4.168300,18.838334,0.000000,31.898103,57.099015,-4.168300,-1.563104
2023-11-27,1.36341,-0.475498,-0.869932,6.173313,4.074362,50.952401,-0.047990,0.298664,-91.449387,0.049459,...,-4.879568,0.739363,336.523046,-4.669553,16.064803,2.488806,0.000000,57.099015,-4.074362,-1.553675
2023-11-28,1.36156,-3.112423,-0.574886,5.272969,4.173783,40.776027,-0.047990,0.298664,-95.822011,0.049459,...,-12.643956,1.768419,336.523046,-4.610167,133.279795,-11.002999,265.593769,57.099015,-4.173783,-0.985320
2023-11-29,1.35716,0.475221,0.619694,7.249218,7.249218,68.704425,-0.047990,0.298664,-84.620294,0.049459,...,-10.405028,1.387253,336.523046,-1.974853,116.750746,-10.649080,206.836057,57.099015,0.218593,-0.861414
2023-11-30,1.35887,-0.609518,-0.752164,5.272969,7.814067,65.839414,-0.047990,0.298664,-94.128909,0.049459,...,-9.888854,1.117328,336.523046,-3.051105,103.824386,-29.179151,170.945595,28.433618,-2.213209,-0.859817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,1.35032,0.075055,0.103258,3.542563,5.953425,73.902806,-0.962137,-27.968892,-94.294447,-0.868408,...,-3.007269,0.337193,909.462456,-3.508327,45.349402,-60.426459,13.407579,98.612786,-0.759477,0.207799
2024-08-27,1.34869,0.290639,0.265004,3.850410,5.047632,69.491264,-0.965645,-26.437978,-99.684090,-0.907135,...,-3.097691,0.347235,970.865201,-3.625909,43.035299,-107.668869,11.598583,43.233748,-0.446875,0.138326
2024-08-28,1.34436,-0.181574,-0.046450,3.850410,5.805650,77.311069,-0.912093,-15.058530,-86.393939,-0.747985,...,-2.184085,0.378684,985.128441,-1.902726,40.287461,-137.388680,10.764050,40.151236,1.211498,0.061855


In [14]:
import json
feature_names = X_df.columns
with open('USDCAD_D1_3112_BuyNEW/feature_names.json', 'w') as f:
    json.dump(list(feature_names), f)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from joblib import dump
import os
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

sum_fp = 0
sum_tp = 0

# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Directory to save model and scaler
os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)

# Initialise RandomForestClassifier with fixed hyperparameters
rf_classifier_mt = RandomForestClassifier(
    n_estimators=200,
    max_depth=40,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=0,
    max_leaf_nodes=20,
    bootstrap=True,
    oob_score=True,
    ccp_alpha=0,
    class_weight={0: 10, 1: 15}
)

# Ensure the time series order is preserved in the split
X = X_df.iloc[:, :-1].values
y = X_df['b_flag'].values

for fold, (train_index, test_index) in enumerate(tscv.split(X_df)):
    print(f"Fold {fold + 1}")

    # Train data
    x_train, y_train = X[train_index], y[train_index]
    # Test data
    x_test, y_test = X[test_index], y[test_index]

    # Scale Data
    sc_mt = StandardScaler()
    x_train = sc_mt.fit_transform(x_train)
    x_test = sc_mt.transform(x_test)

    # Save scaler
    dump(sc_mt, f'Dump_GBPUSD_D1_3112_Buy/scaler_fold_{fold + 1}.joblib')

    # Train Model
    rf_classifier_mt.fit(x_train, y_train)

    # Save the trained model
    dump(rf_classifier_mt, f'Dump_GBPUSD_D1_3112_Buy/model_fold_{fold + 1}.joblib')

    # Predict on training data
    y_train_pred = rf_classifier_mt.predict(x_train)

    print("Confusion Matrix (Training Data):")
    cm_train = confusion_matrix(y_train, y_train_pred)
    print(cm_train)

    train_false_positives = cm_train[0][1]
    train_true_positives = cm_train[1][1]

    train_precision = precision_score(y_train, y_train_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, rf_classifier_mt.predict_proba(x_train)[:, 1])

    print('Training Data Results:')
    print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
    print('False Positives:', train_false_positives)
    print('True Positives:', train_true_positives)
    print('Precision:', train_precision)
    print('Accuracy:', train_accuracy)
    print('Recall:', train_recall)
    print('F1 Score:', train_f1)
    print('ROC AUC:', train_roc_auc)
    print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
    print('BreakEvenRatio:', round(BreakEvenRatio, 2))
    print('____________________________________________________________________________________________________________________________')

    # Predict on testing data
    y_test_pred = rf_classifier_mt.predict(x_test)

    print("Confusion Matrix (Testing Data):")
    cm_test = confusion_matrix(y_test, y_test_pred)
    print(cm_test)

    test_false_positives = cm_test[0][1]
    test_true_positives = cm_test[1][1]

    test_precision = precision_score(y_test, y_test_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_roc_auc = roc_auc_score(y_test, rf_classifier_mt.predict_proba(x_test)[:, 1])

    print('Testing Data Results:')
    print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
    print('False Positives:', test_false_positives)
    print('True Positives:', test_true_positives)
    print('Precision:', test_precision)
    print('Accuracy:', test_accuracy)
    print('Recall:', test_recall)
    print('F1 Score:', test_f1)
    print('ROC AUC:', test_roc_auc)
    print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
    print('BreakEvenRatio:', round(BreakEvenRatio, 2))
    print('____________________________________________________________________________________________________________________________')


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from joblib import dump
import os
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

# Ensure reproducibility
random_state = 0

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Initialize accumulators for overall metrics
overall_metrics = {
    'train': {'precision_sum': 0, 'accuracy_sum': 0, 'recall_sum': 0, 'f1_sum': 0, 'roc_auc_sum': 0},
    'test': {'precision_sum': 0, 'accuracy_sum': 0, 'recall_sum': 0, 'f1_sum': 0, 'roc_auc_sum': 0},
}

best_model = None
best_score = float('-inf')

fold_idx = 1
for train_index, test_index in tscv.split(X_df):
    train_data, test_data = X_df.iloc[train_index], X_df.iloc[test_index]

    # Train data
    x_train = train_data.iloc[:, :-1].values
    y_train = train_data['b_flag'].values
    # Test data
    x_test = test_data.iloc[:, :-1].values
    y_test = test_data['b_flag'].values

    # Scale Data
    sc_mt = StandardScaler()
    x_train = sc_mt.fit_transform(x_train)
    x_test = sc_mt.transform(x_test)

    os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)
    dump(sc_mt, 'Dump_GBPUSD_D1_3112_Buy/scaler.joblib')

    # Initialize RandomForestClassifier with fixed hyperparameters
    rf_classifier_mt = RandomForestClassifier(
        n_estimators=200,
        max_depth=40,
        min_samples_split=2,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=random_state,
        max_leaf_nodes=20,
        bootstrap=True,
        oob_score=True,
        ccp_alpha=0,
        class_weight={0:10, 1:15}
    )

    # Train Model
    rf_classifier_mt.fit(x_train, y_train)

    dump(rf_classifier_mt, f'Dump_GBPUSD_D1_3112_Buy/model_fold_{fold_idx}.joblib')

    # Predict on training data
    y_train_pred = rf_classifier_mt.predict(x_train)

    cm_train = confusion_matrix(y_train, y_train_pred)
    train_false_positives = cm_train[0][1]
    train_true_positives = cm_train[1][1]

    train_precision = precision_score(y_train, y_train_pred)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, rf_classifier_mt.predict_proba(x_train)[:, 1])

    overall_metrics['train']['precision_sum'] += train_precision
    overall_metrics['train']['accuracy_sum'] += train_accuracy
    overall_metrics['train']['recall_sum'] += train_recall
    overall_metrics['train']['f1_sum'] += train_f1
    overall_metrics['train']['roc_auc_sum'] += train_roc_auc

    # Predict on testing data
    y_test_pred = rf_classifier_mt.predict(x_test)

    cm_test = confusion_matrix(y_test, y_test_pred)
    test_false_positives = cm_test[0][1]
    test_true_positives = cm_test[1][1]

    test_precision = precision_score(y_test, y_test_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_roc_auc = roc_auc_score(y_test, rf_classifier_mt.predict_proba(x_test)[:, 1])

    overall_metrics['test']['precision_sum'] += test_precision
    overall_metrics['test']['accuracy_sum'] += test_accuracy
    overall_metrics['test']['recall_sum'] += test_recall
    overall_metrics['test']['f1_sum'] += test_f1
    overall_metrics['test']['roc_auc_sum'] += test_roc_auc

    if test_f1 > best_score:
        best_score = test_f1
        best_model = rf_classifier_mt
        best_fold_idx = fold_idx

        if fold_idx == 4:
            test_predictions = pd.DataFrame({
                'time': test_data.index,
                'actual': y_test,
                'predicted': y_test_pred
            })
            test_predictions.to_csv('test_predictions_fold_4.csv', index=False)

    fold_idx += 1

# Average metrics across all folds
avg_train_precision = overall_metrics['train']['precision_sum'] / tscv.get_n_splits()
avg_train_accuracy = overall_metrics['train']['accuracy_sum'] / tscv.get_n_splits()
avg_train_recall = overall_metrics['train']['recall_sum'] / tscv.get_n_splits()
avg_train_f1 = overall_metrics['train']['f1_sum'] / tscv.get_n_splits()
avg_train_roc_auc = overall_metrics['train']['roc_auc_sum'] / tscv.get_n_splits()

avg_test_precision = overall_metrics['test']['precision_sum'] / tscv.get_n_splits()
avg_test_accuracy = overall_metrics['test']['accuracy_sum'] / tscv.get_n_splits()
avg_test_recall = overall_metrics['test']['recall_sum'] / tscv.get_n_splits()
avg_test_f1 = overall_metrics['test']['f1_sum'] / tscv.get_n_splits()
avg_test_roc_auc = overall_metrics['test']['roc_auc_sum'] / tscv.get_n_splits()

print("Average Metrics Across All Folds (Training):")
print('Precision:', avg_train_precision)
print('Accuracy:', avg_train_accuracy)
print('Recall:', avg_train_recall)
print('F1 Score:', avg_train_f1)
print('ROC AUC:', avg_train_roc_auc)
print('____________________________________________________________________________________________________________________________')
print("Average Metrics Across All Folds (Testing):")
print('Precision:', avg_test_precision)
print('Accuracy:', avg_test_accuracy)
print('Recall:', avg_test_recall)
print('F1 Score:', avg_test_f1)
print('ROC AUC:', avg_test_roc_auc)
print('____________________________________________________________________________________________________________________________')
print(f'Best model found in fold {best_fold_idx} with F1 score: {best_score}')

# Save the best model
best_model_path = 'Dump_GBPUSD_D1_3112_Buy/best_model.joblib'
dump(best_model, best_model_path)
print(f'Saved the best model to {best_model_path}')
# Final Training on Full Training Data
final_train_data = X_df
x_final_train = final_train_data.iloc[:, :-1].values
y_final_train = final_train_data['b_flag'].values

# Scale Data
sc_final = StandardScaler()
x_final_train = sc_final.fit_transform(x_final_train)

# Train Final Model
final_rf_classifier = RandomForestClassifier(
    n_estimators=200,
    max_depth=40,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=random_state,
    max_leaf_nodes=20,
    bootstrap=True,
    oob_score=True,
    ccp_alpha=0,
    class_weight={0:10, 1:15}
)

final_rf_classifier.fit(x_final_train, y_final_train)

# Save Final Model and Scaler
os.makedirs('Final_Model_GBPUSD_D1_3112_Buy', exist_ok=True)
dump(sc_final, 'Final_Model_GBPUSD_D1_3112_Buy/scaler_final.joblib')
dump(final_rf_classifier, 'Final_Model_GBPUSD_D1_3112_Buy/final_model.joblib')
print('Final model and scaler saved.')


In [ ]:
from joblib import load
final_model_path = 'Final_Model_GBPUSD_D1_3112_Buy/final_model.joblib'
scaler_path = 'Final_Model_GBPUSD_D1_3112_Buy/scaler_final.joblib'
final_model = load(final_model_path)
scaler = load(scaler_path)

test_data = X_df

x_test = test_data.iloc[:, :-1].values
y_test = test_data['b_flag'].values

# Scale Data
x_test = scaler.transform(test_data)

y_test_pred = rf_classifier_mt.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, rf_classifier_mt.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

In [ ]:
""" from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from joblib import dump
import os
import pandas as pd
import numpy as np
sum_fp = 0
sum_tp = 0


# Ensure the time series order is preserved in the split
split = int(0.90 * len(X_df))
train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['b_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['b_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Buy/scaler.joblib')

# Define hyperparameters for tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialise RandomForestClassifier with GridSearchCV
rf_classifier_mt = RandomForestClassifier(random_state=27)
grid_search = GridSearchCV(estimator=rf_classifier_mt, param_grid=param_grid, cv=5, scoring='accuracy')

# Train Model with GridSearchCV
grid_search.fit(x_train, y_train)
best_rf_classifier_mt = grid_search.best_estimator_

dump(best_rf_classifier_mt, 'Dump_GBPUSD_D1_3112_Buy/model.joblib')

# Predict on training data
y_train_pred = best_rf_classifier_mt.predict(x_train)

print("Confusion Matrix (Training Data):")
cm_train = confusion_matrix(y_train, y_train_pred)
print(cm_train)

train_false_positives = cm_train[0][1]
train_true_positives = cm_train[1][1]

train_precision = precision_score(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, best_rf_classifier_mt.predict_proba(x_train)[:, 1])

print('Training Data Results:')
print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
print('False Positives:', train_false_positives)
print('True Positives:', train_true_positives)
print('Precision:', train_precision)
print('Accuracy:', train_accuracy)
print('Recall:', train_recall)
print('F1 Score:', train_f1)
print('ROC AUC:', train_roc_auc)
print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

# Predict on testing data
y_test_pred = best_rf_classifier_mt.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, best_rf_classifier_mt.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________') """

In [ ]:
import MetaTrader5 as mt
import pandas as pd
import numpy as np
import talib
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
from sklearn.feature_selection import RFE
import os

# Initialize MetaTrader
mt.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login, password, server)

# Get data
symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2015, 1, 1), datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

# Shift the entire DataFrame down by one row
shifted_df = df.copy()
shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']] = shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']].shift(1)

df = shifted_df

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

# Label the data
StopLoss = 1
TakeProfit = 2
BreakEvenRatio = StopLoss / (StopLoss + TakeProfit)
label_data(df, [StopLoss], [TakeProfit], 80, symbol, False)

# Feature extraction
df.drop(columns=['s_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['b_flag'])
X_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]

# Addressing Collinearity
correlation_matrix = X_df.drop(columns=['b_flag']).corr()
threshold = 0.9
high_corr_pairs = np.where(np.abs(correlation_matrix) > threshold)
high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                   for x, y in zip(*high_corr_pairs) if x != y and x < y]

def remove_high_corr_features(corr_pairs, data):
    to_drop = set()
    for (feature1, feature2) in corr_pairs:
        to_drop.add(feature1)  # You can choose to drop feature1 or feature2 based on domain knowledge or preference
    return data.drop(columns=list(to_drop))

X_df_reduced = remove_high_corr_features(high_corr_pairs, X_df.drop(columns=['b_flag']))

# Apply RFE to select top features
clf = RandomForestClassifier(n_estimators=100, random_state=27)
rfe = RFE(estimator=clf, n_features_to_select=10)  # Adjust the number of features to select
rfe.fit(X_df_reduced, X_df['b_flag'])

X_rfe_selected = X_df_reduced.iloc[:, rfe.get_support()]
X_rfe_selected['b_flag'] = X_df['b_flag'].values

# Ensure the time series order is preserved in the split
split = int(0.90 * len(X_rfe_selected))
train_data, test_data = X_rfe_selected.iloc[:split], X_rfe_selected.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['b_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['b_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Buy/scaler.joblib')

# Initialise RandomForestClassifier with fixed hyperparameters
rf_classifier_mt = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=27
)

# Train Model
rf_classifier_mt.fit(x_train, y_train)

dump(rf_classifier_mt, 'Dump_GBPUSD_D1_3112_Buy/model.joblib')

# Predict on training data
y_train_pred = rf_classifier_mt.predict(x_train)

print("Confusion Matrix (Training Data):")
cm_train = confusion_matrix(y_train, y_train_pred)
print(cm_train)

train_false_positives = cm_train[0][1]
train_true_positives = cm_train[1][1]

train_precision = precision_score(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, rf_classifier_mt.predict_proba(x_train)[:, 1])

print('Training Data Results:')
print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
print('False Positives:', train_false_positives)
print('True Positives:', train_true_positives)
print('Precision:', train_precision)
print('Accuracy:', train_accuracy)
print('Recall:', train_recall)
print('F1 Score:', train_f1)
print('ROC AUC:', train_roc_auc)
print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

# Predict on testing data
y_test_pred = rf_classifier_mt.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, rf_classifier_mt.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


In [ ]:
import MetaTrader5 as mt
import pandas as pd
import numpy as np
import talib
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
import os

# Initialize MetaTrader
mt.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login, password, server)

# Get data
symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

# Shift the entire DataFrame down by one row
shifted_df = df.copy()
shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']] = shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']].shift(1)

df = shifted_df

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

# Label the data
StopLoss = 1
TakeProfit = 2
BreakEvenRatio = StopLoss / (StopLoss + TakeProfit)
label_data(df, [StopLoss], [TakeProfit], 80, symbol, False)

# Feature extraction
df.drop(columns=['s_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['b_flag'])
X_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]

# Addressing Collinearity
def remove_high_corr_features(data, threshold=0.9):
    # Calculate the correlation matrix
    correlation_matrix = data.corr()
    
    # Find pairs of highly correlated features
    high_corr_pairs = np.where(np.abs(correlation_matrix) > threshold)
    high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                       for x, y in zip(*high_corr_pairs) if x != y and x < y]
    
    # Function to remove one feature from each pair of highly correlated features
    to_drop = set()
    for (feature1, feature2) in high_corr_pairs:
        to_drop.add(feature1)  # You can choose to drop feature1 or feature2 based on domain knowledge or preference
    
    # Drop the identified features
    return data.drop(columns=list(to_drop))

# Apply the function to remove highly correlated features
X_df_reduced = remove_high_corr_features(X_df.drop(columns=['b_flag']), threshold=0.8)

# Ensure the time series order is preserved in the split
split = int(0.80 * len(X_df_reduced))
train_data, test_data = X_df_reduced.iloc[:split], X_df_reduced.iloc[split:]

# Train data
x_train = train_data.values
y_train = X_df.loc[train_data.index, 'b_flag'].values
# Test data
x_test = test_data.values
y_test = X_df.loc[test_data.index, 'b_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Buy/scaler.joblib')

# Hyperparameter tuning using GridSearchCV with TimeSeriesSplit
param_grid = {
    'rfe__n_features_to_select': [5, 10, 15],  # RFE hyperparameters
    'clf__n_estimators': [50, 100, 150],
    'clf__max_depth': [10, 20, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__random_state': [0, 27, 42]  # Add multiple random states
}

# Create a pipeline
pipeline = Pipeline([
    ('rfe', RFE(estimator=RandomForestClassifier(), step=1)),  # RFE with default estimator
    ('clf', RandomForestClassifier())  # Classifier
])

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=tscv, scoring='f1', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Retrieve the best model from grid search
best_model = grid_search.best_estimator_

# Train Model
best_model.fit(x_train, y_train)

# Save the model
dump(best_model, 'Dump_GBPUSD_D1_3112_Buy/model.joblib')

# Predict on training data
y_train_pred = best_model.predict(x_train)

print("Confusion Matrix (Training Data):")
cm_train = confusion_matrix(y_train, y_train_pred)
print(cm_train)

train_false_positives = cm_train[0][1]
train_true_positives = cm_train[1][1]

train_precision = precision_score(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, best_model.predict_proba(x_train)[:, 1])

print('Training Data Results:')
print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
print('False Positives:', train_false_positives)
print('True Positives:', train_true_positives)
print('Precision:', train_precision)
print('Accuracy:', train_accuracy)
print('Recall:', train_recall)
print('F1 Score:', train_f1)
print('ROC AUC:', train_roc_auc)
print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

# Predict on testing data
y_test_pred = best_model.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, best_model.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


In [ ]:
import MetaTrader5 as mt
import pandas as pd
import numpy as np
import talib
from joblib import dump
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
import os
from own_functions import *

# Initialize MetaTrader
mt.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login, password, server)

# Get data
symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2015, 1, 1), datetime.now()))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df ['close'], timeperiod=145)

# Shift the entire DataFrame down by one row
shifted_df = df.copy()
shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']] = shifted_df[['open', 'high', 'low', 'close', 'WILLR_15', 'WILLR_23', 'WILLR_42', 'WILLR_145']].shift(1)

df = shifted_df

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

# Label the data
StopLoss = 1
TakeProfit = 2
BreakEvenRatio = StopLoss / (StopLoss + TakeProfit)
label_data(df, [StopLoss], [TakeProfit], 80, symbol, False)

# Feature extraction
df.drop(columns=['s_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['b_flag'])
X_df = X_df[[col for col in X_df if col != 'b_flag'] + ['b_flag']]

# Addressing Collinearity
def remove_high_corr_features(data, threshold=0.8):
    # Calculate the correlation matrix
    correlation_matrix = data.corr()
    
    # Find pairs of highly correlated features
    high_corr_pairs = np.where(np.abs(correlation_matrix) > threshold)
    high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                       for x, y in zip(*high_corr_pairs) if x != y and x < y]
    
    # Function to remove one feature from each pair of highly correlated features
    to_drop = set()
    for (feature1, feature2) in high_corr_pairs:
        to_drop.add(feature1)  # You can choose to drop feature1 or feature2 based on domain knowledge or preference
    
    # Drop the identified features
    return data.drop(columns=list(to_drop))

# Apply the function to remove highly correlated features
X_df_reduced = remove_high_corr_features(X_df.drop(columns=['b_flag']), threshold=0.8)

# Ensure the time series order is preserved in the split
split = int(0.80 * len(X_df_reduced))
train_data, test_data = X_df_reduced.iloc[:split], X_df_reduced.iloc[split:]

# Train data
x_train = train_data.values
y_train = X_df.loc[train_data.index, 'b_flag'].values
# Test data
x_test = test_data.values
y_test = X_df.loc[test_data.index, 'b_flag'].values

# Scale Data
sc_mt = StandardScaler()
x_train = sc_mt.fit_transform(x_train)
x_test = sc_mt.transform(x_test)

os.makedirs('Dump_GBPUSD_D1_3112_Buy', exist_ok=True)
dump(sc_mt, 'Dump_GBPUSD_D1_3112_Buy/scaler.joblib')

# Hyperparameter tuning using GridSearchCV with TimeSeriesSplit
param_grid = {
    'rfe__n_features_to_select': [5, 10, 15],  # RFE hyperparameters
    'clf__n_estimators': [50, 100, 150],
    'clf__max_depth': [10, 20, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__random_state': [0, 27, 42]  # Add multiple random states
}

# Create a pipeline
pipeline = Pipeline([
    ('rfe', RFE(estimator=RandomForestClassifier(), step=1)),  # RFE with default estimator
    ('clf', RandomForestClassifier())  # Classifier
])

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

class LoggingGridSearchCV(GridSearchCV):
    def fit(self, X, y=None, **fit_params):
        print("Starting GridSearchCV...")
        super().fit(X, y, **fit_params)
        means = self.cv_results_['mean_test_score']
        stds = self.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, self.cv_results_['params']):
            print(f"Mean: {mean:.3f}, Std: {std:.3f}, Params: {params}")
        print("GridSearchCV complete.")

grid_search = LoggingGridSearchCV(estimator=pipeline, param_grid=param_grid, cv=tscv, scoring='f1', n_jobs=-1)
grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Retrieve the best model from grid search
best_model = grid_search.best_estimator_

# Train Model
best_model.fit(x_train, y_train)

# Save the model
dump(best_model, 'Dump_GBPUSD_D1_3112_Buy/model.joblib')

# Predict on training data
y_train_pred = best_model.predict(x_train)

print("Confusion Matrix (Training Data):")
cm_train = confusion_matrix(y_train, y_train_pred)
print(cm_train)

train_false_positives = cm_train[0][1]
train_true_positives = cm_train[1][1]

train_precision = precision_score(y_train, y_train_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, best_model.predict_proba(x_train)[:, 1])

print('Training Data Results:')
print('WIN/LOSS-Diff:', round(100 * (train_precision - BreakEvenRatio), 2), '%')
print('False Positives:', train_false_positives)
print('True Positives:', train_true_positives)
print('Precision:', train_precision)
print('Accuracy:', train_accuracy)
print('Recall:', train_recall)
print('F1 Score:', train_f1)
print('ROC AUC:', train_roc_auc)
print('Ratio Total:', round(100 * (train_true_positives / (train_false_positives + train_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')

# Predict on testing data
y_test_pred = best_model.predict(x_test)

print("Confusion Matrix (Testing Data):")
cm_test = confusion_matrix(y_test, y_test_pred)
print(cm_test)

test_false_positives = cm_test[0][1]
test_true_positives = cm_test[1][1]

test_precision = precision_score(y_test, y_test_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, best_model.predict_proba(x_test)[:, 1])

print('Testing Data Results:')
print('WIN/LOSS-Diff:', round(100 * (test_precision - BreakEvenRatio), 2), '%')
print('False Positives:', test_false_positives)
print('True Positives:', test_true_positives)
print('Precision:', test_precision)
print('Accuracy:', test_accuracy)
print('Recall:', test_recall)
print('F1 Score:', test_f1)
print('ROC AUC:', test_roc_auc)
print('Ratio Total:', round(100 * (test_true_positives / (test_false_positives + test_true_positives)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


In [ ]:
print(high_corr_pairs)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

# Function to plot confusion matrix
def plot_confusion_matrix(cm, title):
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    plt.show()

# Plot Confusion Matrix for Training Data
plot_confusion_matrix(cm_train, "Confusion Matrix (Training Data)")

# Plot Confusion Matrix for Testing Data
plot_confusion_matrix(cm_test, "Confusion Matrix (Testing Data)")

# Metrics to plot
metrics = {
    'Precision': (train_precision, test_precision),
    'Accuracy': (train_accuracy, test_accuracy),
    'Recall': (train_recall, test_recall),
    'F1 Score': (train_f1, test_f1),
    'ROC AUC': (train_roc_auc, test_roc_auc)
}

# Function to plot metrics
def plot_metrics(metrics):
    train_metrics = [value[0] for value in metrics.values()]
    test_metrics = [value[1] for value in metrics.values()]
    metric_names = list(metrics.keys())

    x = np.arange(len(metric_names))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(12, 8))
    rects1 = ax.bar(x - width/2, train_metrics, width, label='Training')
    rects2 = ax.bar(x + width/2, test_metrics, width, label='Testing')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xlabel('Metrics')
    ax.set_title('Performance Metrics (Training vs Testing)')
    ax.set_xticks(x)
    ax.set_xticklabels(metric_names)
    ax.legend()

    # Attach a text label above each bar in *rects*, displaying its height.
    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(round(height, 2)),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    autolabel(rects1)
    autolabel(rects2)

    fig.tight_layout()

    plt.show()

# Plot Performance Metrics
plot_metrics(metrics)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# Function to plot learning curves
def plot_learning_curves(train_errors, test_errors, param_range, param_name):
    plt.figure(figsize=(12, 8))
    plt.plot(param_range, train_errors, 'o-', color='r', label='Training MSE')
    plt.plot(param_range, test_errors, 'o-', color='g', label='Testing MSE')
    plt.title(f'Learning Curve ({param_name} vs MSE)')
    plt.xlabel(param_name)
    plt.ylabel('Mean Squared Error')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

# Lists to store the errors
train_errors = []
test_errors = []

# Range of estimators to evaluate
n_estimators_range = [10, 50, 100, 200, 300, 400, 500]

# Evaluate the model performance for different number of estimators
for n_estimators in n_estimators_range:
    rf_classifier_mt = RandomForestClassifier(n_estimators=n_estimators, random_state=27)
    rf_classifier_mt.fit(x_train, y_train)

    # Predict on training data
    y_train_pred = rf_classifier_mt.predict(x_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_errors.append(train_mse)

    # Predict on testing data
    y_test_pred = rf_classifier_mt.predict(x_test)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_errors.append(test_mse)

# Plot learning curves
plot_learning_curves(train_errors, test_errors, n_estimators_range, 'Number of Estimators')


In [ ]:
df_pred = pd.DataFrame(index=test_data.index)
df_pred['prediction'] = y_pred
df_pred.to_csv('predictGBPUSD_D1_3112_Buy.csv')